In [1]:
# 필요한 모듈 및 라이브러리 불러오기
from selenium.webdriver import Chrome
import time
import sqlite3
from pandas.io import sql
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import random
import sys
import re
import requests
from bs4 import BeautifulSoup
import datetime
import warnings
warnings.filterwarnings(action='ignore')

## 음료 데이터 정리

In [3]:
# 데이터 불러오기
drink = pd.read_csv('/Users/JGH/Python/Section Project 4/drink.csv')

In [4]:
# 불필요하다고 생각하는 상품 제거
drink.drop(drink[drink.category =='스타벅스 주스(병음료)'].index, inplace=True)

In [5]:
drink

,id,name,category,calories(kcal),sugars(g),protein(g),sodium(mg),saturated_fat(mg),caffeine(mg),allergy
0,1001,리저브 나이트로,콜드 브루 커피,5,0,0.0,0.0,0.0,190,NaN
1,1002,나이트로 바닐라 크림,콜드 브루 커피,80,10,1.0,40.0,2.0,232,우유
2,1003,나이트로 콜드 브루,콜드 브루 커피,5,0,0.0,5.0,0.0,245,NaN
3,1004,돌체 콜드 브루,콜드 브루 커피,265,29,8.0,130.0,9.0,155,우유
4,1005,리저브 콜드 브루,콜드 브루 커피,5,0,0.0,0.0,0.0,190,NaN
...,...,...,...,...,...,...,...,...,...,...
119,1120,스타벅스 슬래머,기타 제조 음료,235,55,0.0,0.0,0.0,0,NaN
120,1121,스팀 우유,기타 제조 음료,215,18,13.0,173.0,7.0,0,우유
121,1122,아이스 제주 까망 라떼,기타 제조 음료,440,45,10.0,210.0,11.0,0,땅콩 / 대두 / 우유 / 알류 / 밀 / 오징어
122,1123,우유,기타 제조 음료,240,18,12.0,200.0,8.0,0,우유


In [13]:
# 데이터베이스에 적합하도록 음료수의 알러지 데이터는 따로 관리
drink_db = drink.iloc[:,:9]

In [2]:
drink_db= pd.read_csv('/Users/JGH/Python/Section Project 4/drink_db.csv')

In [4]:
# 음료의 상태(차가움,뜨거움)을 나타내는 칼럼 추가
drink_db['status']=np.nan
ice_rows1 = drink_db[drink_db['name'].str.contains('아이스')].index.tolist()
ice_rows2 = drink_db[(drink_db['category'] == '블렌디드') | (drink_db['category'] == '프라푸치노')].index.tolist()
ice_rows = list(set(ice_rows1).union(ice_rows2))
drink_db.loc[ice_rows, 'status'] = 'Ice'
drink_db.loc[~drink_db.index.isin(ice_rows), 'status'] = 'Hot'

In [53]:
# 가격은 공식 홈페이지에서 제공되지 않기 때문에, 저장 후 직접 찾아서 파일에 추가
drink_db.to_csv('/Users/JGH/Python/Section Project 4/drink_db.csv', index=False)

In [54]:
drink_db= pd.read_csv('/Users/JGH/Python/Section Project 4/drink_db.csv')

In [18]:
drink_allergy_info = drink.iloc[:,[0,1,9]]

In [20]:
drink_allergy = pd.DataFrame(columns=['id', 'name', 'allergy'])

# 기존의 drink_allergy 데이터프레임에서 알러지 정보를 추출하여 새로운 테이블에 추가
for index, row in drink_allergy_info.iterrows():
    allergies = row['allergy']
    if pd.notnull(allergies):  # 알러지 정보가 있는 경우에만 처리
        allergy_list = allergies.split(' / ')
        for allergy in allergy_list:
            new_row = {
                'id': row['id'],
                'name': row['name'],
                'allergy': allergy
            }
            drink_allergy = drink_allergy.append(new_row, ignore_index=True)

# 결과 출력
drink_allergy

,id,name,allergy
0,1002,나이트로 바닐라 크림,우유
1,1004,돌체 콜드 브루,우유
2,1007,바닐라 크림 콜드 브루,우유
3,1010,제주 비자림 콜드 브루,우유
4,1013,콜드 브루 몰트,대두
...,...,...,...
107,1124,제주 까망 라떼,대두
108,1124,제주 까망 라떼,우유
109,1124,제주 까망 라떼,알류
110,1124,제주 까망 라떼,밀


In [21]:
drink_allergy.to_csv('/Users/JGH/Python/Section Project 4/drink_allergy.csv', index=False)

# 고객 데이터 생성 및 정리

In [149]:
import pandas as pd
import numpy as np

sex_list = ['남성', '여성']
age_list = ['20대 미만', '20대', '30대', '40대', '50대', '60대 이상']
allergy_list = [np.nan, '우유', '복숭아', '땅콩', '땅콩 / 복숭아']

customer = pd.DataFrame(columns=['id', 'sex', 'age', 'allergy'])

customer_count = 1000
customer_id = 10001

# 성별 비율 설정 (남성: 38%, 여성: 62%)
sex_ratio = [38, 62]

# 연령 비율 설정 (20대 미만: 10%, 20대: 24%, 30대: 22%, 40대: 18%, 50대: 15%, 60대 이상: 11%)
age_ratio = [10, 24, 22, 18, 15, 11]

# 알러지 비율 설정 (없음: 57%, 우유: 3%, 복숭아: 15%, 땅콩: 20%, 땅콩 / 복숭아: 5%)
allergy_ratio = [57, 3, 15, 20, 5]

for _ in range(customer_count):
    # 성별 랜덤 선택
    sex = np.random.choice(sex_list, p=[ratio / sum(sex_ratio) for ratio in sex_ratio])

    # 연령 랜덤 선택
    age = np.random.choice(age_list, p=[ratio / sum(age_ratio) for ratio in age_ratio])

    # 알러지 랜덤 선택
    allergy = np.random.choice(allergy_list, p=[ratio / sum(allergy_ratio) for ratio in allergy_ratio])

    data = {
        'id': customer_id,
        'sex': sex,
        'age': age,
        'allergy': allergy
    }

    customer = customer.append(data, ignore_index=True)
    customer_id += 1

# 결과 출력
customer

,id,sex,age,allergy
0,10001,여성,20대 미만,nan
1,10002,남성,50대,땅콩
2,10003,여성,50대,nan
3,10004,여성,30대,nan
4,10005,여성,30대,복숭아
...,...,...,...,...
995,10996,여성,30대,nan
996,10997,여성,40대,nan
997,10998,여성,50대,nan
998,10999,여성,60대 이상,nan


In [150]:
customer.to_csv('/Users/JGH/Python/Section Project 4/customer.csv', index=False)

In [151]:
customer_db = customer.iloc[:,:3]
customer_db.to_csv('/Users/JGH/Python/Section Project 4/customer_db.csv', index=False)

In [152]:
customer_allergy_info = customer.iloc[:,[0,3]]
customer_allergy_info.drop(customer_allergy_info[customer_allergy_info.allergy == 'nan'].index, inplace=True)
customer_allergy = pd.DataFrame(columns=['id','allergy'])
for index, row in customer_allergy_info.iterrows():
    allergies = row['allergy']
    if pd.notnull(allergies):
        allergy_list = allergies.split(' / ')
        for allergy in allergy_list:
            new_row = {
                'id': row['id'],
                'allergy': allergy
            }
            customer_allergy = customer_allergy.append(new_row, ignore_index=True)

# 결과 출력
customer_allergy

,id,allergy
0,10002,땅콩
1,10005,복숭아
2,10006,땅콩
3,10007,복숭아
4,10009,복숭아
...,...,...
493,10991,복숭아
494,10992,땅콩
495,10994,땅콩
496,10995,복숭아


In [153]:
customer_allergy.to_csv('/Users/JGH/Python/Section Project 4/customer_allergy.csv', index=False)

# 주문 데이터

In [193]:
order = pd.DataFrame(columns=['id', 'customer_id', 'product_id','product_name','date'])

In [194]:
order

,id,customer_id,product_id,product_name,date


In [195]:
customer_db= pd.read_csv('/Users/JGH/Python/Section Project 4/customer_db.csv')
drink_db= pd.read_csv('/Users/JGH/Python/Section Project 4/drink_db.csv')
customer_allergy= pd.read_csv('/Users/JGH/Python/Section Project 4/customer_allergy.csv')
drink_allergy= pd.read_csv('/Users/JGH/Python/Section Project 4/drink_allergy.csv')

In [196]:
start_date = datetime.date(2022, 1, 1)
end_date = datetime.date(2022, 12, 31)

order = pd.DataFrame(columns=['id', 'customer_id', 'drink_id', 'drink_name', 'date'])

current_date = start_date
while current_date <= end_date:
    if current_date.month >= 5 and current_date.month <= 9:
        num_per_day = random.randint(300, 700)
        weights = {
            '에스프레소': 40,
            '티': 2,
            '콜드 브루 커피': 6,
            '블렌디드': 25,
            '프라푸치노': 25,
            '기타 제조 음료': 2,
        }
        stat_weights = {
            'Ice' : 90,
            'Hot' : 10
        }
    else:
        num_per_day = random.randint(100, 350)
        weights = {
            '에스프레소': 40,
            '티': 10,
            '콜드 브루 커피': 10,
            '블렌디드': 15,
            '프라푸치노': 15,
            '기타 제조 음료': 2.5,
            '스타벅스 리프레셔': 2.5,
            '스타벅스 피지오': 2.5,
            '브루드 커피': 2.5
        }
        stat_weights = {
            'Ice' : 60,
            'Hot' : 40
        }

    for _ in range(num_per_day):
        category = random.choices(list(weights.keys()), weights=list(weights.values()))[0]
        drinks_in_category = drink_db[drink_db['category'] == category]
        if category == '에스프레소':
            espresso_drinks = ['아이스 카페 아메리카노', '카페 아메리카노', '기타']
            es_drink = random.choices(espresso_drinks, weights=[60, 10, 30])[0]
            if es_drink == '기타':
                status = random.choices(list(stat_weights.keys()), weights=list(stat_weights.values()))[0]
                drinks_in_status = drinks_in_category[drinks_in_category['status'] == status]
                drink = random.choice(drinks_in_status['name'].to_list())
            else:
                drink = es_drink           
        
        else:
            drink = random.choice(drinks_in_category['name'].to_list())

        customer = random.choice(customer_db['id'].to_list())

        d_allergy = drink_allergy[drink_allergy.name == drink].allergy.to_list()
        c_allergy = customer_allergy[customer_allergy.id == customer].allergy.to_list()
        if not any(allergy in d_allergy for allergy in c_allergy):
            data = {
                'id': 1,
                'customer_id': customer,
                'drink_id': drink_db[drink_db.name == drink].id.values[0],
                'drink_name': drink,
                'date': current_date
            }

            order = order.append(data, ignore_index=True)

    current_date += datetime.timedelta(days=1)


In [197]:
order['id'] = order.index + 10000001

In [198]:
order

,id,customer_id,drink_id,drink_name,date
0,10000001,10034,1059,제주 유기농 말차로 만든 크림 프라푸치노,2022-01-01
1,10000002,10318,1019,아이스 카페 아메리카노,2022-01-01
2,10000003,10527,1054,자바 칩 프라푸치노,2022-01-01
3,10000004,10235,1033,아이스 라벤더 카페 브레베,2022-01-01
4,10000005,10673,1064,라이트 키위 라임 블렌디드,2022-01-01
...,...,...,...,...,...
120178,10120179,10246,1060,초콜릿 크림 칩 프라푸치노,2022-12-31
120179,10120180,10042,1030,사케라또 비안코 오버 아이스,2022-12-31
120180,10120181,10711,1019,아이스 카페 아메리카노,2022-12-31
120181,10120182,10672,1019,아이스 카페 아메리카노,2022-12-31


In [199]:
order.to_csv('/Users/JGH/Python/Section Project 4/order_db.csv', index=False)